# Testing with the noise

In [5]:
import pennylane as qml
import numpy as np

nqubits = 6
dev = qml.device("default.mixed", wires=nqubits)

@qml.qnode(dev)
def kernel_circ_noisy(a, b, noise_prob=0.01):
    qml.AmplitudeEmbedding(a, wires=range(nqubits), pad_with=0, normalize=True)

    for w in range(nqubits):
        qml.DepolarizingChannel(noise_prob, wires=w)

    
    qml.adjoint(qml.AmplitudeEmbedding(b, wires=range(nqubits), pad_with=0, normalize=True))

    
    for w in range(nqubits):
        qml.DepolarizingChannel(noise_prob, wires=w)

    return qml.probs(wires=range(nqubits))


In [8]:
def qkernel_test_noisy(X_test, X_train, noise_prob=0.01):
    K_test = np.zeros((len(X_test), len(X_train)))
    for i, a in enumerate(X_test):
        print(f"Row {i+1}/{len(X_test)}")
        for j, b in enumerate(X_train):
            if j <= i:  
                K_test[i, j] = K_test[j, i]
                continue
            K_test[i, j] = kernel_circ_noisy(a, b, noise_prob=noise_prob)[0]
    np.save("quantum_kernel_test_noisy.npy", K_test)
    return K_test

In [ ]:
from sklearn.svm import SVC

K_train = np.load("quantum_kernel.npy")
y_train = np.load("labels.npy")
X_train = np.load("train_data.npy")
X_test = np.load("test_data.npy")

svm = SVC(kernel="precomputed")
svm.fit(K_train, y_train)

K_test = qkernel_test_noisy(X_test, X_train, noise_prob=0.02) 
predictions = svm.predict(K_test)

print("Predictions on noisy test set:", predictions)


Row 1/152
Row 2/152
Row 3/152
Row 4/152
Row 5/152
Row 6/152
Row 7/152
Row 8/152
Row 9/152
Row 10/152
Row 11/152
Row 12/152
Row 13/152
Row 14/152
Row 15/152
Row 16/152
Row 17/152
Row 18/152
Row 19/152
Row 20/152
Row 21/152
Row 22/152
Row 23/152
Row 24/152
Row 25/152
Row 26/152
Row 27/152
Row 28/152
Row 29/152
Row 30/152
Row 31/152
Row 32/152
Row 33/152
Row 34/152
Row 35/152
Row 36/152
Row 37/152
Row 38/152
Row 39/152
Row 40/152
Row 41/152
Row 42/152
Row 43/152
Row 44/152
Row 45/152
Row 46/152
Row 47/152
Row 48/152
Row 49/152
Row 50/152
Row 51/152
Row 52/152
Row 53/152


# Testing without the noise

In [ ]:
svm = SVC(kernel="precomputed")
svm.fit(K_train, y_train)

In [ ]:
predictions = svm.predict(K_train)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_train, predictions))
print(classification_report(y_train, predictions))